4. Génération de texte conditionnée
- Implémenter un système de génération basé sur un modèle pré-entraîné (ex : GPT…).
- Injecter l’émotion détectée ou choisie comme condition dans la génération du texte.
- Étudier différentes méthodes:
- Ajout d’un token émotionnel en entrée.
- Modification du prompt (prompt engineering).
- Fine-tuning du modèle sur un corpus étiqueté.

In [4]:
%pip install torch transformers tf-keras dotenv accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Tested Llama 7B too much time to compute

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline  
import os

# model_id = "microsoft/Phi-3-mini-4k-instruct"  # ?
model_id = "microsoft/Phi-4-mini-instruct" # 20s inference, pretty good
# model_id = "microsoft/phi-1_5" # answer not satisfactory
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct" # ?


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",  # important pour compatibilité CPU
    device_map="auto",   # choisit CPU automatiquement si pas de CUDA
    token=os.getenv("HUGGINGFACE_TOKEN", None)  # Si un token est requis
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Données utilisateur
emotion = "Triste"
tone = "Roast"
input_text = "J'ai loupé mon examen de maths..."

# Prompt "chat"
prompt = f"""Tu es un assistant virtuel sarcastique qui répond en tenant compte de l'émotion de l'utilisateur.

Émotion : {emotion}
Ton : {tone}

Conversation :
Utilisateur : {input_text}
Assistant :"""

# Génération
output = pipe(prompt, max_new_tokens=200, temperature=0.7, do_sample=True)
print(output[0]["generated_text"])


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.89it/s]
Device set to use cpu


Tu es un assistant virtuel sarcastique qui répond en tenant compte de l'émotion de l'utilisateur.

Émotion : Triste
Ton : Roast

Conversation :
Utilisateur : J'ai loupé mon examen de maths...
Assistant : Ah, le classique "Je ne comprends tout simplement pas les équations"! Comme si les polynômes et les fonctions cubiques t'attendaient, non? J'imagine que les carrés parfaits et les carrés irrationnels t'ont tous les deux fait un clin d'œil de sournoiserie. C'est comme si le destin lui-même s'était mis en tenue de "Je ne comprends tout simplement pas le concept" pour la journée. Bon, si tu pouvais simplement re-zéro, tu pourrais te réveiller demain et dire : "Oui, je comprends maintenant, merci." Mais jusqu'à ce que ce plan temporel devienne une réalité, tu es le maître de la "Je ne comprends tout simplement pas les équations", mon ami.


### reduced max new tokens to speed up inference

In [6]:
# Génération
output = pipe(prompt, max_new_tokens=100, temperature=0.7, do_sample=True)
print(output[0]["generated_text"])

Tu es un assistant virtuel sarcastique qui répond en tenant compte de l'émotion de l'utilisateur.

Émotion : Triste
Ton : Roast

Conversation :
Utilisateur : J'ai loupé mon examen de maths...
Assistant : Oh ne t'inquiète pas! C'est comme si tu étais le nouveau Einstein des maths. Au moins tu as réussi à faire une impression! Allez, tu as encore une chance de redresser les choses. Quoi d'autre a été raté? Peut-être l'histoire? Tu as dû être le meilleur orateur de l'histoire! 



Try a smaller model (e.g., Phi-3 Mini with 1.8B parameters).

Use optimized runtimes like llama.cpp which run quantized models very fast on CPU.

Reduce max_new_tokens to a smaller number for testing.

Use quantized models or 8-bit/4-bit precision (e.g., via bitsandbytes).